In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from transformers import pipeline
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import xmltodict, json
import ollama
import pandas as pd


#cik = '000320193' # apple inc CIK

#url = f"https://www.sec.gov/files/company_tickers.json"# to find the company CIK from its ticker (e.g., APPL CIK is 0000320193)
#search_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=8-K&count={count}&output=atom" # to find 8-Ks from CIK

# cik = '000320193' # AAPL

def fetch_cik(ticker):

    driver = webdriver.Chrome()

    # URL to fetch the JSON data
    url = "https://www.sec.gov/files/company_tickers.json"
    cik = None
    # Open the URL
    driver.get(url)

    # Wait for the page to load (you can adjust the time as needed)
    # time.sleep(5)  # Wait for 5 seconds to ensure the page is loaded

    # Get the page source (HTML)
    page_source = driver.find_element(By.TAG_NAME, 'pre').text

    try:
        # Extract JSON data from page source
        json_data = json.loads(page_source)

    except json.JSONDecodeError:
        print("Error decoding the JSON response.")
    for d in json_data:
        if ticker == json_data[d]['ticker']:
            cik = json_data[d]['cik_str']
    # Close the browser
    driver.quit()
    return cik



def fetch_8k_filings(ticker, cik, count):

    driver = webdriver.Chrome()

    # URL to fetch the JSON data
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=8-K&count={count}&output=atom"
    cik = None
    # Open the URL
    driver.get(url)

    # Wait for the page to load (you can adjust the time as needed)
    # time.sleep(5)  # Wait for 5 seconds to ensure the page is loaded

    # Get the page source (HTML)
    page_source = driver.find_element(By.TAG_NAME, 'pre').text
    output_data = []

    try:
        # Extract JSON data from page source
        xml_data = xmltodict.parse(str(page_source))
        json_data = json.loads(json.dumps(xml_data))
        for entry in json_data['feed']['entry']:
            output_data.append(
                {
                    "Company Name": json_data['feed']['company-info']['conformed-name'],
                    "Stock Name": ticker,
                    "Filing Time": entry['content']['filing-date'],
                    "New Product": "",
                    "Product Description": "",
                    "link": entry['content']['filing-href']
                }
            )

    except json.JSONDecodeError:
        print("Error decoding the JSON response.")
    # Close the browser
    driver.quit()
    return output_data



#filings = fetch_8k_filings(ticker, cik, 10)
#filings






   




In [39]:
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext
def get_after_substring(text, substring):
    """
    Returns the part of the string after the first occurrence of a substring.
    If the substring is not found, it returns an empty string.
    """
    index = text.find(substring)
    if index != -1:
        return text[index + len(substring):]
    return ""

In [50]:
tickers = ["AAPL", "TSLA"]
all_filings = []
options = webdriver.ChromeOptions()
options.add_argument("--enable-javascript")
driver = webdriver.Chrome(options=options)
for ticker in tickers:

    cik = fetch_cik(ticker)

    filings = fetch_8k_filings(ticker, cik, 75)

    for i in range(len(filings)):
       # if i != 0:
          #  continue
        try: 


            url = filings[i]['link']

            print(url)
            driver.get(url)
            product_name = driver.find_element(By.XPATH, '//*[@id="contentDiv"]/div[1]/div[2]/div[3]/div[2]').text
            page_source = driver.find_element(By.XPATH, '//*[@id="contentDiv"]/div[2]/div/table/tbody/tr[4]/td[3]/a').get_attribute('href')
            print(page_source)
            driver.get(page_source)
            #time.sleep(5)
            file = driver.find_element(By.TAG_NAME, 'body').text
            cleaned_file = cleanhtml(file)
            response = ollama.chat(model='deepseek-r1:1.5b', messages=[
                {'role': 'user', 'content': f'summarize this SEC 8-K document for {filings[i]['Company Name']}: {cleaned_file}'},
            ])
            print(response['message']['content'])
            filings[i]['Product Description'] = get_after_substring(response['message']['content'],"</think>").replace(",","").replace("\n","")[:160]
            filings[i]['New Product'] = product_name
        except:
            continue
    all_filings.extend(filings)
driver.quit()
print(all_filings)

output_csv = []
for i in range(len(all_filings)):
    output_csv.append(
                {
                    "Company Name": all_filings[i]['Company Name'],
                    "Stock Name": all_filings[i]['Stock Name'],
                    "Filing Time": all_filings[i]['Filing Time'],
                    "New Product": all_filings[i]['New Product'],
                    "Product Description": all_filings[i]['Product Description']
                }
            )

df = pd.DataFrame(output_csv)
df.to_csv('filename.csv', index=False)




Exception ignored in: <function Service.__del__ at 0x000001B875AD9300>
Traceback (most recent call last):
  File "C:\Users\Robbie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\selenium\webdriver\common\service.py", line 200, in __del__
    self.stop()
  File "C:\Users\Robbie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\selenium\webdriver\common\service.py", line 157, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\Robbie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\selenium\webdriver\common\service.py", line 137, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\urllib\request.py", line 215, in ur

https://www.sec.gov/Archives/edgar/data/320193/000114036125005876/0001140361-25-005876-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000162828025002993/0001628280-25-002993-index.htm
[{'Company Name': 'Apple Inc.', 'Stock Name': 'AAPL', 'Filing Time': '2025-02-25', 'New Product': '', 'Product Description': '', 'link': 'https://www.sec.gov/Archives/edgar/data/320193/000114036125005876/0001140361-25-005876-index.htm'}, {'Company Name': 'Apple Inc.', 'Stock Name': 'AAPL', 'Filing Time': '2025-01-30', 'New Product': '', 'Product Description': '', 'link': 'https://www.sec.gov/Archives/edgar/data/320193/000032019325000007/0000320193-25-000007-index.htm'}, {'Company Name': 'Apple Inc.', 'Stock Name': 'AAPL', 'Filing Time': '2025-01-03', 'New Product': '', 'Product Description': '', 'link': 'https://www.sec.gov/Archives/edgar/data/320193/000114036125000228/0001140361-25-000228-index.htm'}, {'Company Name': 'Apple Inc.', 'Stock Name': 'AAPL', 'Filing Time': '2024-10-31', 'New Produc

In [25]:
import subprocess

def query_ollama(model, prompt):
    """Runs an Ollama query using a specified model and prompt."""
    try:
        # Execute the Ollama command and capture output
        result = subprocess.run(
            ["ollama", "run", model, prompt],
            capture_output=True, text=True#False if true gives you an error
        )
        return result.stdout.strip()
    except Exception as e:
        return f"Error: {e}"

if __name__ == "__main__":
    model_name = "deepseek-r1:1.5b"  # Change this if needed
    print(f"Querying Ollama with model: {model_name}")

    question = "I am a UCF Fintech MS. student, what do you suggest i learn to improve my career?"

    response = query_ollama(model_name, question)
    print("\nOllama Response:\n", response)


Querying Ollama with model: deepseek-r1:1.5b

Ollama Response:
 Error: [WinError 2] The system cannot find the file specified


In [34]:
import ollama
response = ollama.chat(model='deepseek-r1:1.5b', messages=[
        {'role': 'user', 'content': f'just return yes or no: {cleanhtml(file)}'},
    ])
print(response['message']['content'])
response = ollama.chat(model='deepseek-r1:1.5b', messages=[
        {'role': 'user', 'content': f'summarize this SEC 8-K document for Apple in 160 characters: {cleanhtml(file)}'},
    ])

print(response['message']['content'])

<think>
Okay, so I'm trying to figure out how to return "yes" in this situation. Let me start by understanding the context provided. The user has a bunch of lines that look like SEC filings or financial reports from a company, probably related to investors or analysts looking at their financial performance and news events.

Each line starts with "dei", which likely stands for "decimal," maybe an abbreviation used in some financial data formats or reporting conventions. Then there's the word "Notes Due" followed by something like "%" and a number in parentheses, such as 2.000% Notes due 2027. These seem like notes that were set to be paid out at certain points in time.

I notice that some lines have more detailed information, including the source of data ("Source:"), context ("Context:"), and specific identifiers or references (e.g., "False"). Others are shorter, just with a date or type, like "False" or "2031." The user's query is straightforward: "just return yes."

Now, thinking abou